In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

INFO 05-19 21:57:54 [__init__.py:239] Automatically detected platform cuda.


In [2]:
import torch
model_path = "google/gemma-3-27b-it"
llm = LLM(model_path, max_model_len=32768, dtype=torch.bfloat16, gpu_memory_utilization=0.99)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 05-19 21:58:51 [config.py:689] This model supports multiple tasks: {'classify', 'embed', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
INFO 05-19 21:58:51 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-19 21:59:00 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='google/gemma-3-27b-it', speculative_config=None, tokenizer='google/gemma-3-27b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=N

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


INFO 05-19 21:59:24 [gpu_model_runner.py:1276] Starting to load model google/gemma-3-27b-it...
INFO 05-19 21:59:24 [config.py:3466] cudagraph sizes specified by model runner [1, 2, 4, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512] is overridden by config [512, 384, 256, 128, 4, 2, 1, 392, 264, 136, 8, 400, 272, 144, 16, 408, 280, 152, 24, 416, 288, 160, 32, 424, 296, 168, 40, 432, 304, 176, 48, 440, 312, 184, 56, 448, 320, 192, 64, 456, 328, 200, 72, 464, 336, 208, 80, 472, 344, 216, 88, 120, 480, 352, 248, 224, 96, 488, 504, 360, 232, 104, 496, 368, 240, 112, 376]
WARNING 05-19 21:59:24 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best 

Loading safetensors checkpoint shards:   0% Completed | 0/12 [00:00<?, ?it/s]


INFO 05-19 21:59:40 [loader.py:458] Loading weights took 8.69 seconds
INFO 05-19 21:59:40 [gpu_model_runner.py:1291] Model loading took 51.4482 GiB and 15.990654 seconds
INFO 05-19 21:59:40 [gpu_model_runner.py:1560] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 64 image items of the maximum feature size.
INFO 05-19 22:00:01 [backends.py:416] Using cache directory: /nfs/home/tanz/.cache/vllm/torch_compile_cache/12e98ecf6e/rank_0_0 for vLLM's torch.compile
INFO 05-19 22:00:01 [backends.py:426] Dynamo bytecode transform time: 18.95 s
INFO 05-19 22:00:04 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 05-19 22:00:33 [monitor.py:33] torch.compile takes 18.95 s in total
INFO 05-19 22:00:35 [kv_cache_utils.py:634] GPU KV cache size: 62,096 tokens
INFO 05-19 22:00:35 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 1.90x
INFO 05-19 22:01:00 [gpu_model_runner.py:1626] Graph capturing finished i

# Zero-shot

In [14]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": """# 社交网络文本本地人身份界定标准多标签分析

## 1. 任务目标

分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词）**的显著标准是什么，并依据本手册RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。

**核心限定:** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。

## 2. 核心原则

*   **聚焦作者核心论点:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、关于“本地人”身份界定的核心观点、主张或评价。理解作者**想说什么**是首要任务。
*   **分析认同逻辑的功能角色:** 识别出的认同逻辑元素不仅仅是孤立的特征，更要理解它们在作者构建其核心论点时所扮演的**功能角色**。
*   **主动推理语境与隐含逻辑:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据或其评价标准。
*   **排除抵抗标准与无关讨论:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**。对于未与“本地人”身份界定直接关联的讨论，**不予标注**。

## 3. 认同逻辑框架

**统一句型模板:**
"作者认为，人们（或作者本人）应该/常常/可以凭借[认同逻辑类型]所代表的逻辑或标准，来界定谁是/不是‘本地人’，或者评价一个‘本地’区域的好坏/范围。"

**认同逻辑类型定义与特征:**

*   **认同逻辑1：Vernacular Spatial Authority（社会习惯性区域认知）**
    *   **定义:** 作者认同或描述的、当地人对城市内部空间类别的共同、习惯或历史沉淀的认知。这些认知并非基于官方的行政边界，而是反映了集体的情感图谱，被用作分配评价或象征性标签的文化速记。
    *   **核心辨识特征:**
        *   作者明确提出或默认一种关于**哪些区域“算”或“不算”本地核心区、哪些区域之间存在身份或认知差异**的划分逻辑（例如“三环以外不算成都”）。
        *   作者通过描述特定区域的**象征意义、历史标签或社会普遍看法（集体情感地图）**来暗示其在“本地”身份体系中的地位。
        *   作者讨论**“市区范围”、“城市边界”等概念的社会认同变迁或当前共识**。

*   **认同逻辑2：Administrative Legitimacy（行政归属合法性）**
    *   **定义:** 对官方管辖权、法律地位或行政称谓的诉求。这类发言者以户口登记（一种户籍制度）、行政区合并或市镇重新划分为由，证明自己被纳入或被排除在外。
    *   **核心辨识特征:** 关键词包括“行政区域划分”、“划进”、“归属”、“户口是哪的”、“身份证开头”等，作为判断某人/某地在行政法理上是否算“本地”的依据。

*   **认同逻辑3：Family Rootedness（家族历史根基/个体成长史）**
    *   **定义:** 作者认同或引用的、根据家族定居的世代深度来评估当地的合法性。该类别中的主张强调血统、祖先或与该地区的长期家族联系。
    *   **核心辨识特征:**
        *   强调“**土生土长**”、“**世代**”、“**祖辈**”、“**父辈**”、“**三代以内**”、“**从小就/幼儿园就来了**”等，以此证明某人是“根正苗红的本地人”或已形成“本地人”的身份认同基础。
        *   描述因迁移历史（或缺乏迁移史）导致的身份差异。

*   **认同逻辑4：Linguistic-Cultural Recognition（文化语言识别性）**
    *   **定义:** 依赖方言、口音或文化语言习惯作为边界标志。地区性语言特点被视为内部人地位的代名词，偏差往往会引起嘲笑或不信任。也可能包括对特定地方文化习惯（如习俗、生活方式）的认同。
    *   **核心辨识特征:** 提及“讲本地话”、“口音”、“听不懂/受不了某些口音”、“懂不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”或具备“本地属性”的标准。

*   **认同逻辑5：Functional Livability（生活功能便利性与环境品质认知）**
    *   **定义:** 作者认同或引用的、从物质基础设施（如交通、住房、教育或获得服务的途径）的角度对城市地区进行评估，通常是为了宣称空间优越性或可取性。
    *   **核心辨识特征:**
        *   提及“地铁”、“配套”、“方便”、“教育”、“绿化好”、“人少”、“街道界面”、“舒服”等，并**将其与对一个区域是否“好”、是否“宜居”或是否“值得居住”的评价直接关联**。
        *   作者因某地的认同逻辑5特性而将其视为理想的“本地”生活空间，或因缺乏认同逻辑5特性而认为某地不符合“好的本地”标准。

*   **认同逻辑6：Social Embeddedness（社会根基深浅与经济地位象征）**
    *   **定义:** 作者认同或引用的、根据一个人融入当地社会圈子的情况，以及有没有当地的固定资产来判断是不是本地人。这包括社区里的资源（像分红、对社区熟不熟）以及物质或象征性的资源，比如（继承的）房产。
    *   **核心辨识特征:**
        *   提及“人脉”、“分红”、“老关系”、“社区影响力”、“朋友在附近”、“在市中心有几套房”、“房价高/低对身份认同的影响”、“拿出一两百万买房”等，并**明确或隐含地将这些与“本地人身份的稳固性、真实性、层级或对区域的评价标准”挂钩**。

*   **认同逻辑7：Occupational Typification（职业象征性）**
    *   **定义:** 作者认同或引用的、通过将某些地区与占主导地位的职业群体--如公务员、外来务工人员或企业主--联系起来，从而勾勒出阶级和价值的隐性等级。
    *   **核心辨识特征:** 提及特定职业或人群类型（如“农民”、“种地的”、“打工的”），并**将其与特定区域的“本地属性”、居民构成或社会分层强关联**，从而界定身份或评价区域。

## 4. 标注流程

1.  **第一步：识别文本中提及的命名实体**
    *   通读文本，把文中提及的实体记录下来。

2.  **第二步：为每个命名实体匹配论点**
    *   细读文本，为每个提及的实体都匹配上相应的观点表述。

3.  **第三步：分析每个观点表述背后隐藏的认同逻辑**
    *   对每个实体，都分析说话人表达的观点（若有）背后是基于哪一项认同逻辑，可为多个认同逻辑。

4.  **第四步：输出结果**
    *   输出所有匹配到的**一个或多个**认同逻辑类别标签[认同逻辑N]，用 `, ` 分隔。

## 5. 注意事项

*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份或评价“本地”区域的标准，并理解其在作者核心论点中的功能**。
*   不要仅凭关键词进行机械匹配，深入理解作者话语背后的逻辑、意图及其核心观点。**按照 4. 标注流程 中的步骤从第一步一直到第四步地进行思考，不要省略步骤**
*   如果作者仅仅在描述一个现象、表达个人偏好或评价社会地位/财富，而没有流露出自己用这些标准来定义“谁是本地人”或评价“本地区域好坏/范围/区隔”，则**不予标注**。
*   在 `输出：`标志后输出结果。输出结果后**立即停止**，不要再继续输出任何内容。"""
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [15]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 500
sampling_params = SamplingParams(temperature=1.0, top_p=0.95, top_k=64, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            #enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "第一步：识别文本中提及的命名实体"
        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [16]:
import re
import pandas as pd

# Primary regex for "**输出[:：]**"
primary_output_marker_pattern = re.compile(r"\*\*输出[:：]\*\*\s*(.*)", re.DOTALL)
# Secondary regex for "第六步：输出结果"
# This pattern looks for "第六步：输出结果" followed by optional whitespace,
# and then captures the rest of the string. re.DOTALL ensures `.` matches newlines.
secondary_output_marker_pattern = re.compile(r"第六步：输出结果\s*(.*)", re.DOTALL)
# Regex for splitting by "输出:" or "输出：" - used for the simple/plain marker
simple_output_marker_regex = r"输出[:：]"

outputs = all_outputs # Assuming all_outputs is defined from a previous cell
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # Assuming texts is defined from a previous cell
    model_full_output_text = output_obj.outputs[0].text # The raw text from the model's output

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary output marker first (bolded)
    match_primary = primary_output_marker_pattern.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (**输出[:：]**)"
    else:
        # If primary marker is not found, try the secondary marker (step-based)
        match_secondary = secondary_output_marker_pattern.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (第六步：输出结果)"
        else:
            # If secondary marker is not found, try the simple plain "输出[:：]" marker.
            # We split by the marker and take the content after the *last* occurrence.
            parts = re.split(simple_output_marker_regex, model_full_output_text)
            if len(parts) > 1: # Marker was found
                extracted_label_part = parts[-1].strip()
                used_marker_type = "simple (plain '输出[:：]' marker)"
    
    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` `)
        # Also handle cases where the model might output "无" or "（无）" for no labels.
        if not label or label == "无" or label == "（无）":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "无" or "（无）"
                print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if none of the output markers are found
        print(f"Warning: Could not find any of the expected markers: primary ('{primary_output_marker_pattern.pattern}'), secondary ('{secondary_output_marker_pattern.pattern}'), or simple plain ('{simple_output_marker_regex}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_zero_shot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())


Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output:

第二步：为每个命名实体匹配论点
第三步：分析每个观点表述背后隐藏的认同逻辑
第四步：输出结果

这个任务就是让你读别人的社交媒体帖子，然后分析帖子作者**如何定义“本地人”**。

**要点：**

*   **别管帖子说的是什么，只管作者怎么定义“本地人”。** 比如，帖子说“这地方的房价太贵了”，如果你觉得作者是在暗示只有买得起房的人才算本地人，那就可以标注。但如果只是单纯讨论房价，就别管。
*   **重点看作者认同什么，而不是反对什么。**  如果作者说“有些人认为户口决定本地人，但我不同意”，你**不标注**。你要找的是作者认同的定义。
*   **一个帖子可以有多个定义。** 比如，作者可能既说了“祖上三代都在这儿才算本地人”，又说了“得讲本地话才算”，那就要标注两个类别。
*   **仔细理解作者的意思。** 很多时候，作者不会直接说“我认为本地人应该…”，而是通过一些暗示来表达。你需要像侦探一样去推理。

**认同逻辑类别就像是不同类型的定义：**

*   **社会习惯性区域认知:**  “三环以外都不算XX” 这种对区域的划分和认知。
*   **行政归属合法性:**  “户口在哪儿”、“身份证开头是几” 这种基于户籍和行政区划的定义。
*   **家族历史根基/个体成长史:**  “土生土长”、“祖辈就在这儿” 这种强调祖上和成长经历的定义。
*   **文化语言识别性:** “得讲本地话”、“口音” 这种基于语言和文化的定义。
*   **生活功能便利性与环境品质认知:**  “这儿配套好”、“交通方便” 这种基于生活便利程度的定义。
*   

# Few-shot

In [17]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": """# 社交网络文本本地人身份界定标准多标签分析

## 1. 任务目标

分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词）**的显著标准是什么，并依据本手册RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。

**核心限定:** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。

## 2. 核心原则

*   **聚焦作者核心论点:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、关于“本地人”身份界定的核心观点、主张或评价。理解作者**想说什么**是首要任务。
*   **分析认同逻辑的功能角色:** 识别出的认同逻辑元素不仅仅是孤立的特征，更要理解它们在作者构建其核心论点时所扮演的**功能角色**。
*   **主动推理语境与隐含逻辑:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据或其评价标准。
*   **排除抵抗标准与无关讨论:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**。对于未与“本地人”身份界定直接关联的讨论，**不予标注**。

## 3. 认同逻辑框架

**统一句型模板:**
"作者认为，人们（或作者本人）应该/常常/可以凭借[认同逻辑类型]所代表的逻辑或标准，来界定谁是/不是‘本地人’，或者评价一个‘本地’区域的好坏/范围。"

**认同逻辑类型定义与特征:**

*   **认同逻辑1：Vernacular Spatial Authority（社会习惯性区域认知）**
    *   **定义:** 作者认同或描述的、当地人对城市内部空间类别的共同、习惯或历史沉淀的认知。这些认知并非基于官方的行政边界，而是反映了集体的情感图谱，被用作分配评价或象征性标签的文化速记。
    *   **核心辨识特征:**
        *   作者明确提出或默认一种关于**哪些区域“算”或“不算”本地核心区、哪些区域之间存在身份或认知差异**的划分逻辑（例如“三环以外不算成都”）。
        *   作者通过描述特定区域的**象征意义、历史标签或社会普遍看法（集体情感地图）**来暗示其在“本地”身份体系中的地位。
        *   作者讨论**“市区范围”、“城市边界”等概念的社会认同变迁或当前共识**。

*   **认同逻辑2：Administrative Legitimacy（行政归属合法性）**
    *   **定义:** 对官方管辖权、法律地位或行政称谓的诉求。这类发言者以户口登记（一种户籍制度）、行政区合并或市镇重新划分为由，证明自己被纳入或被排除在外。
    *   **核心辨识特征:** 关键词包括“行政区域划分”、“划进”、“归属”、“户口是哪的”、“身份证开头”等，作为判断某人/某地在行政法理上是否算“本地”的依据。

*   **认同逻辑3：Family Rootedness（家族历史根基/个体成长史）**
    *   **定义:** 作者认同或引用的、根据家族定居的世代深度来评估当地的合法性。该类别中的主张强调血统、祖先或与该地区的长期家族联系。
    *   **核心辨识特征:**
        *   强调“**土生土长**”、“**世代**”、“**祖辈**”、“**父辈**”、“**三代以内**”、“**从小就/幼儿园就来了**”等，以此证明某人是“根正苗红的本地人”或已形成“本地人”的身份认同基础。
        *   描述因迁移历史（或缺乏迁移史）导致的身份差异。

*   **认同逻辑4：Linguistic-Cultural Recognition（文化语言识别性）**
    *   **定义:** 依赖方言、口音或文化语言习惯作为边界标志。地区性语言特点被视为内部人地位的代名词，偏差往往会引起嘲笑或不信任。也可能包括对特定地方文化习惯（如习俗、生活方式）的认同。
    *   **核心辨识特征:** 提及“讲本地话”、“口音”、“听不懂/受不了某些口音”、“懂不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”或具备“本地属性”的标准。

*   **认同逻辑5：Functional Livability（生活功能便利性与环境品质认知）**
    *   **定义:** 作者认同或引用的、从物质基础设施（如交通、住房、教育或获得服务的途径）的角度对城市地区进行评估，通常是为了宣称空间优越性或可取性。
    *   **核心辨识特征:**
        *   提及“地铁”、“配套”、“方便”、“教育”、“绿化好”、“人少”、“街道界面”、“舒服”等，并**将其与对一个区域是否“好”、是否“宜居”或是否“值得居住”的评价直接关联**。
        *   作者因某地的认同逻辑5特性而将其视为理想的“本地”生活空间，或因缺乏认同逻辑5特性而认为某地不符合“好的本地”标准。

*   **认同逻辑6：Social Embeddedness（社会根基深浅与经济地位象征）**
    *   **定义:** 作者认同或引用的、根据一个人融入当地社会圈子的情况，以及有没有当地的固定资产来判断是不是本地人。这包括社区里的资源（像分红、对社区熟不熟）以及物质或象征性的资源，比如（继承的）房产。
    *   **核心辨识特征:**
        *   提及“人脉”、“分红”、“老关系”、“社区影响力”、“朋友在附近”、“在市中心有几套房”、“房价高/低对身份认同的影响”、“拿出一两百万买房”等，并**明确或隐含地将这些与“本地人身份的稳固性、真实性、层级或对区域的评价标准”挂钩**。

*   **认同逻辑7：Occupational Typification（职业象征性）**
    *   **定义:** 作者认同或引用的、通过将某些地区与占主导地位的职业群体--如公务员、外来务工人员或企业主--联系起来，从而勾勒出阶级和价值的隐性等级。
    *   **核心辨识特征:** 提及特定职业或人群类型（如“农民”、“种地的”、“打工的”），并**将其与特定区域的“本地属性”、居民构成或社会分层强关联**，从而界定身份或评价区域。

## 4. 标注流程

1.  **第一步：识别文本中提及的命名实体**
    *   通读文本，把文中提及的实体记录下来。

2.  **第二步：为每个命名实体匹配论点**
    *   细读文本，为每个提及的实体都匹配上相应的观点表述。

3.  **第三步：分析每个观点表述背后隐藏的认同逻辑**
    *   对每个实体，都分析说话人表达的观点（若有）背后是基于哪一项认同逻辑，可为多个认同逻辑。

4.  **第四步：输出结果**
    *   输出所有匹配到的**一个或多个**认同逻辑类别标签[认同逻辑N]，用 `, ` 分隔。

## 5. 注意事项

*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份或评价“本地”区域的标准，并理解其在作者核心论点中的功能**。
*   不要仅凭关键词进行机械匹配，深入理解作者话语背后的逻辑、意图及其核心观点。**按照 4. 标注流程 中的步骤从第一步一直到第四步地进行思考，不要省略步骤**
*   如果作者仅仅在描述一个现象、表达个人偏好或评价社会地位/财富，而没有流露出自己用这些标准来定义“谁是本地人”或评价“本地区域好坏/范围/区隔”，则**不予标注**。
*   在 `输出：`标志后输出结果。输出结果后**立即停止**，不要再继续输出任何内容。"""
    },
    {
      "role": "user",
      "content": "崇州的嘛，我们身边的人被问到是哪里的，都直接说崇州，不会说是成都的。从小到大耳濡目染的就是成都人都觉得三环以外的不算成都的。"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 崇州（地名）  \n- 成都（三环以内、三环以外，地理区分）\n\n第二步：为每个命名实体匹配论点  \n- 崇州：作者和身边的人被问时会说自己是“崇州”的，不说“成都的”。\n- 成都（三环以内/以外）：从小的感受和集体认知是“成都人都觉得三环以外的不算成都的”。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 崇州和成都区隔的说法，是在表达一种社会习惯性区域认知——“崇州”虽行政上归成都，但在社会文化习惯上并不认作“成都”本地；而“三环以外不算成都”，则是成都本地社会内部广泛认同的一种空间界定方式，即以“三环”为核心地理象征分割“本地/非本地”。\n- 这里“从小到大耳濡目染的就是成都人都觉得三环以外的不算成都的”，直接体现了集体的空间认知共识，不是官方行政意义上的划分，而是文化情感层面的分区。\n- 没有提及其他如家族根基、行政归属等标准。\n\n第四步：输出结果  \n输出：认同逻辑1"
    },
    {
      "role": "user",
      "content": "抱抱，反正在我眼里我只看行政区域划分，属于青岛那就是青岛人，管是胶州还是城阳，早些年因为交通不便利人们觉得隔着远就罢了，现在不管地铁还是开车一会就到了呀"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 青岛  \n- 胶州  \n- 城阳  \n\n第二步：为每个命名实体匹配论点  \n- 青岛：作者只看行政区域划分，属于青岛的就是青岛人。\n- 胶州、城阳：只要行政上属于青岛，这两个地方的人也算青岛人。作者提及“早些年因交通不便利…现在…地铁开车一会就到了”，作为对空间距离认知变化的补充，但核心标准是行政区域。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 作者明确表达“只看行政区域划分”，即以官方行政归属为唯一认同标准。\n- 也提及“交通便利性”导致社会认知的变化，但该部分用于说明过去与现在的物理距离观念、心理距离的缩短，但并没有表明作者用交通便利性作为界定“本地人”的标准——这仍是对空间观念变化的观察，核心不是用来界定“本地人”身份。\n- 没有体现对家族根基、方言文化、社会根基、职业类型等其他逻辑的认同。\n\n第四步：输出结果  \n输出：认同逻辑2"
    },
    {
      "role": "user",
      "content": "在广州三代以内上就算真正广州人，二代以内是新广州人，一代开荒牛是广州移民，三代都不讲广州话的是外省广州人（祖宗是捞佬）来广州的。"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 三代以内的广州人  \n- 二代以内的新广州人  \n- 一代“开荒牛”（广州移民）  \n- 三代都不讲广州话的人\n\n第二步：为每个命名实体匹配论点  \n- 三代以内：被认为是“真正广州人”，强调家族扎根年限。\n- 二代以内：被认为是“新广州人”，家族定居时间稍短于“真正广州人”。\n- 一代开荒牛：被称为“广州移民”，代表迁入第一代。\n- 三代不讲广州话者：即使家族已经在广州三代，若不讲广州话，被归为“外省广州人”；描述其祖先来自外省。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 对家族在地历史长度（“几代人”）作为评价“本地人”层级的主要依据，典型的家族历史根基认同逻辑。\n- “三代都不讲广州话”则进一步用语言/文化融入作为认同标准，强调语言（广州话）作为身份的象征和界定标志。\n- 没有明显提及行政归属、区域认知、社会关系、职业等标准。\n\n第四步：输出结果  \n输出：认同逻辑3, 认同逻辑4"
    },
    {
      "role": "user",
      "content": "我是成都土著，我觉得还好！说实话哈，只是确确实实有一点受不了一些口音。"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 成都土著（作者自称）\n\n第二步：为每个命名实体匹配论点  \n- 作者自认为“成都土著”，表达自己对“土著”身份的认同。\n- 还提到“有一点受不了一些口音”，表达对某些非本地口音的不适。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- “成都土著”体现出家族或个体的本地根基，但未详细展开代际等家族深度，因此直接家族历史根基的认同逻辑表征不明显。\n- 主要着重于“受不了一些口音”，即用口音、语言作为区分“本地人/非本地人”的实际体验，表达了语言/文化识别是自己衡量本地性的隐含标准。\n\n第四步：输出结果  \n输出：认同逻辑4"
    },
    {
      "role": "user",
      "content": "所以我喜欢北辰，老城区往新区的过渡，街道界面没老城区那么破旧，离市中心又只需要十几分钟，配套啥都有，住起来太舒服哒"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 北辰（区域名）\n- 老城区\n- 新区\n- 市中心\n\n第二步：为每个命名实体匹配论点  \n- 北辰受到作者喜欢，理由包括街道界面较新、离市中心近、配套设施齐全、居住感受好。\n- 与老城区对比，北辰街道界面更好，设施更完善。\n- 市中心——地理位置优势（交通便利）。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 作者对北辰的喜欢是基于生活便利度、配套设施以及居住环境的舒适度进行评价。\n\n第四步：输出结果  \n输出：认同逻辑5"
    },
    {
      "role": "user",
      "content": "毕竟好多家庭不见得可以拿出一两百万买一个环境好点、还在二环内的房子。毕竟我爸妈在成都工作这么久，基本都在一二环内，朋友也在附近，买到郊区肯定不现实，买一二环的房子改造是最好的方案了。"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 一二环内的房子（二环内、地段）  \n- 郊区  \n- 成都（整体城市范围）\n\n第二步：为每个命名实体匹配论点  \n- 一二环内的房子：环境好、价格高（需一两百万）、与作者父母的工作、朋友关系网等密切相关，认为这是最好的选择。\n- 郊区：相较之下认为“买到郊区肯定不现实”，即不符合理想。\n- 作者父母长期在一二环内工作、朋友都在附近，社会关系根基也在一二环内。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 强调“工作久”“朋友在附近”、在市区有足够经济实力买房等社会关系、经济能力，是对社会根基和经济地位的一种展示。\n- 也提到一二环“环境好”，表达出生活便利性、居住环境优越的功能性评价。\n- 没有直接涉及家族历史、行政归属、语言文化、职业类型等逻辑。\n\n第四步：输出结果  \n输出：认同逻辑5, 认同逻辑6"
    },
    {
      "role": "user",
      "content": "李沧黄岛那里是农村，很多种地的农民，老一辈不认为他们是城里人。"
    },
    {
      "role": "assistant",
      "content": "第一步：识别文本中提及的命名实体  \n- 李沧、黄岛（区域名）\n- 农民（种地的人）\n- 老一辈（判断者）\n\n第二步：为每个命名实体匹配论点  \n- 李沧、黄岛被描述为“农村”，居民多为“种地的农民”。\n- “老一辈不认为他们是城里人”，即按照某些标准、身份界定逻辑，不把这些地区的人视为真正的“本地城里人”。\n\n第三步：分析每个观点表述背后隐藏的认同逻辑  \n- 作者提及“种地的农民”，通过职业类别与区域身份强相关地进行区隔，“老一辈”用主要职业类型来划分“是不是城里人”。\n- 还体现了区域空间（农村/城市）的社会集体习惯性认知，但此句重心偏向职业类型与集体观念的结合。\n- 没有直接用行政归属、家族根基、语言文化等逻辑进行界定。\n\n第四步：输出结果  \n输出：认同逻辑7, 认同逻辑1"
    },
    {
      "role": "user",
      "content": input_text
    },
  ]

In [18]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 500
sampling_params = SamplingParams(temperature=1.0, top_p=0.95, top_k=64, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "第一步：识别文本中提及的命名实体  \n"

        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [19]:
import re
import pandas as pd

# Primary regex for "**输出[:：]**"
primary_output_marker_pattern = re.compile(r"\*\*输出[:：]\*\*\s*(.*)", re.DOTALL)
# Secondary regex for "第六步：输出结果"
# This pattern looks for "第六步：输出结果" followed by optional whitespace,
# and then captures the rest of the string. re.DOTALL ensures `.` matches newlines.
secondary_output_marker_pattern = re.compile(r"第六步：输出结果\s*(.*)", re.DOTALL)
# Regex for splitting by "输出:" or "输出：" - used for the simple/plain marker
simple_output_marker_regex = r"输出[:：]"

outputs = all_outputs # Assuming all_outputs is defined from a previous cell
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # Assuming texts is defined from a previous cell
    model_full_output_text = output_obj.outputs[0].text # The raw text from the model's output

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary output marker first (bolded)
    match_primary = primary_output_marker_pattern.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (**输出[:：]**)"
    else:
        # If primary marker is not found, try the secondary marker (step-based)
        match_secondary = secondary_output_marker_pattern.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (第六步：输出结果)"
        else:
            # If secondary marker is not found, try the simple plain "输出[:：]" marker.
            # We split by the marker and take the content after the *last* occurrence.
            parts = re.split(simple_output_marker_regex, model_full_output_text)
            if len(parts) > 1: # Marker was found
                extracted_label_part = parts[-1].strip()
                used_marker_type = "simple (plain '输出[:：]' marker)"
    
    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` `)
        # Also handle cases where the model might output "无" or "（无）" for no labels.
        if not label or label == "无" or label == "（无）":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "无" or "（无）"
                print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if none of the output markers are found
        print(f"Warning: Could not find any of the expected markers: primary ('{primary_output_marker_pattern.pattern}'), secondary ('{secondary_output_marker_pattern.pattern}'), or simple plain ('{simple_output_marker_regex}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_few_shot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())

Successfully processed 483 model outputs.
Results saved to gemma-3-27b-it_few_shot.csv

First 5 rows of the parsed data:
                                 Original_Input_Text  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那...   
1  真的很多，我就是周边的，不管现实还是网上看的很多，他们天天说自己土著然后说我们弯脚杆，碰到过...   
2  我是都江堰的，其他地方的我不敢说，但是我可是从小被成都口音嘲笑哦，我小时候去亲戚家耍过暑假，...   
3  那我说都江堰话被笑的更多，我以前被介绍了个成都人，然后天天给我说他家以前二环以内的，说他家以...   
4  那证明成都人的确不咋地，本来成都话口音离普通话语区就够偏了，很多怪音，他们还歧视本省的人，无...   

                    RL_Types  \
0                       认同逻辑1   
1  认同逻辑1, 认同逻辑4, 认同逻辑6, 认同逻辑3   
2                       认同逻辑4   
3                认同逻辑1, 认同逻辑4   
4                       认同逻辑4   

                                    Raw_Model_Output  
0  - 叙北（区域名）\n- 北门（区域名）\n- 三环（环线）\n- 火北（区域名）\n\n第...  
1  - 周边地区（作者及相关人群所在地）\n- 成都（市中心、二环以内、二环以外）\n- 都江堰...  
2  - 都江堰（作者所在地）\n- 成都（与作者形成对比的“外地”）\n\n第二步：为每个命名实...  
3  - 都江堰话\n- 成都人\n- 二环以内（区域）\n- 宽窄巷子（地标）\n\n第二步：为...  
4  - 成都人\n- 成都话口音\n- 外地人\n- 成都的区县\n- 普通话语区\n\n第二步...  


In [20]:
results_df['RL_Types']

0                           认同逻辑1
1      认同逻辑1, 认同逻辑4, 认同逻辑6, 认同逻辑3
2                           认同逻辑4
3                    认同逻辑1, 认同逻辑4
4                           认同逻辑4
                  ...            
478                         认同逻辑1
479                         认同逻辑1
480                         认同逻辑1
481                  认同逻辑1, 认同逻辑2
482                         认同逻辑1
Name: RL_Types, Length: 483, dtype: object

# No CoT

In [21]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": """# 社交网络文本本地人身份界定标准多标签分析

## 1. 任务目标

分析给定的社交媒体文本片段，判断**作者本人**在其中明确表达认同、或在字里行间**隐含表达认同**的、用于界定**“本地人”（或其近义词）**的显著标准是什么，并依据本手册RL框架，输出所有相应的RL类别标签。一个文本可能符合**一个或多个**RL类别。

**核心限定:** 本任务**仅关注**作者如何界定“本地人”身份。如果文本仅仅讨论财富、社会地位、生活品质、房产优劣、职业好坏等，而**未明确或强烈暗示**这些是用来判断一个人是否为“本地人”的标准，则**不予标注**。

## 2. 核心原则

*   **聚焦作者核心论点:** 您的判断**必须**基于**发言者（即文本作者）**所展现的、关于“本地人”身份界定的核心观点、主张或评价。理解作者**想说什么**是首要任务。
*   **分析认同逻辑的功能角色:** 识别出的认同逻辑元素不仅仅是孤立的特征，更要理解它们在作者构建其核心论点时所扮演的**功能角色**。
*   **主动推理语境与隐含逻辑:** 需要主动进行语境推理，理解话语背后作者认可的“本地人”身份合法性依据或其评价标准。
*   **排除抵抗标准与无关讨论:** 对于作者明确表达抵抗、否定，或仅仅描述他人用来针对自己的标准，**不予标注**。对于未与“本地人”身份界定直接关联的讨论，**不予标注**。

## 3. 认同逻辑框架

**统一句型模板:**
"作者认为，人们（或作者本人）应该/常常/可以凭借[认同逻辑类型]所代表的逻辑或标准，来界定谁是/不是‘本地人’，或者评价一个‘本地’区域的好坏/范围。"

**认同逻辑类型定义与特征:**

*   **认同逻辑1：Vernacular Spatial Authority（社会习惯性区域认知）**
    *   **定义:** 作者认同或描述的、当地人对城市内部空间类别的共同、习惯或历史沉淀的认知。这些认知并非基于官方的行政边界，而是反映了集体的情感图谱，被用作分配评价或象征性标签的文化速记。
    *   **核心辨识特征:**
        *   作者明确提出或默认一种关于**哪些区域“算”或“不算”本地核心区、哪些区域之间存在身份或认知差异**的划分逻辑（例如“三环以外不算成都”）。
        *   作者通过描述特定区域的**象征意义、历史标签或社会普遍看法（集体情感地图）**来暗示其在“本地”身份体系中的地位。
        *   作者讨论**“市区范围”、“城市边界”等概念的社会认同变迁或当前共识**。

*   **认同逻辑2：Administrative Legitimacy（行政归属合法性）**
    *   **定义:** 对官方管辖权、法律地位或行政称谓的诉求。这类发言者以户口登记（一种户籍制度）、行政区合并或市镇重新划分为由，证明自己被纳入或被排除在外。
    *   **核心辨识特征:** 关键词包括“行政区域划分”、“划进”、“归属”、“户口是哪的”、“身份证开头”等，作为判断某人/某地在行政法理上是否算“本地”的依据。

*   **认同逻辑3：Family Rootedness（家族历史根基/个体成长史）**
    *   **定义:** 作者认同或引用的、根据家族定居的世代深度来评估当地的合法性。该类别中的主张强调血统、祖先或与该地区的长期家族联系。
    *   **核心辨识特征:**
        *   强调“**土生土长**”、“**世代**”、“**祖辈**”、“**父辈**”、“**三代以内**”、“**从小就/幼儿园就来了**”等，以此证明某人是“根正苗红的本地人”或已形成“本地人”的身份认同基础。
        *   描述因迁移历史（或缺乏迁移史）导致的身份差异。

*   **认同逻辑4：Linguistic-Cultural Recognition（文化语言识别性）**
    *   **定义:** 依赖方言、口音或文化语言习惯作为边界标志。地区性语言特点被视为内部人地位的代名词，偏差往往会引起嘲笑或不信任。也可能包括对特定地方文化习惯（如习俗、生活方式）的认同。
    *   **核心辨识特征:** 提及“讲本地话”、“口音”、“听不懂/受不了某些口音”、“懂不懂我们这儿的规矩”等，作为判断内外、判断是否为“自己人”或具备“本地属性”的标准。

*   **认同逻辑5：Functional Livability（生活功能便利性与环境品质认知）**
    *   **定义:** 作者认同或引用的、从物质基础设施（如交通、住房、教育或获得服务的途径）的角度对城市地区进行评估，通常是为了宣称空间优越性或可取性。
    *   **核心辨识特征:**
        *   提及“地铁”、“配套”、“方便”、“教育”、“绿化好”、“人少”、“街道界面”、“舒服”等，并**将其与对一个区域是否“好”、是否“宜居”或是否“值得居住”的评价直接关联**。
        *   作者因某地的认同逻辑5特性而将其视为理想的“本地”生活空间，或因缺乏认同逻辑5特性而认为某地不符合“好的本地”标准。

*   **认同逻辑6：Social Embeddedness（社会根基深浅与经济地位象征）**
    *   **定义:** 作者认同或引用的、根据一个人融入当地社会圈子的情况，以及有没有当地的固定资产来判断是不是本地人。这包括社区里的资源（像分红、对社区熟不熟）以及物质或象征性的资源，比如（继承的）房产。
    *   **核心辨识特征:**
        *   提及“人脉”、“分红”、“老关系”、“社区影响力”、“朋友在附近”、“在市中心有几套房”、“房价高/低对身份认同的影响”、“拿出一两百万买房”等，并**明确或隐含地将这些与“本地人身份的稳固性、真实性、层级或对区域的评价标准”挂钩**。

*   **认同逻辑7：Occupational Typification（职业象征性）**
    *   **定义:** 作者认同或引用的、通过将某些地区与占主导地位的职业群体--如公务员、外来务工人员或企业主--联系起来，从而勾勒出阶级和价值的隐性等级。
    *   **核心辨识特征:** 提及特定职业或人群类型（如“农民”、“种地的”、“打工的”），并**将其与特定区域的“本地属性”、居民构成或社会分层强关联**，从而界定身份或评价区域。

## 4. 注意事项

*   时刻提醒自己，标注对象是**作者本人未明确反对的、且用来界定“本地人”身份或评价“本地”区域的标准，并理解其在作者核心论点中的功能**。
*   如果作者仅仅在描述一个现象、表达个人偏好或评价社会地位/财富，而没有流露出自己用这些标准来定义“谁是本地人”或评价“本地区域好坏/范围/区隔”，则**不予标注**。
*   在 `输出：`标志后输出结果。输出结果后**立即停止**，不要再继续输出任何内容。"""
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [22]:
all_outputs = []
batch_size = 500
sampling_params = SamplingParams(temperature=1.0, top_p=0.95, top_k=64, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        )
        formatted_prompts_batch.append(formatted_prompt + "输出：[认同逻辑")
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [23]:
import re
import pandas as pd
# Regex to find the start of the final output section.
# It looks for "输出:" or "输出：" (with either half-width or full-width colon),
# followed by optional whitespace, and then captures the rest of the string (which is the model's raw output).
output_marker_pattern = re.compile(r"输出[:：]\s*(.*)", re.DOTALL) # Removed \*\* to be more flexible
outputs = all_outputs
parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]  # The original text fed into the prompt
    # Construct the string to be parsed. The model's output (output_obj.outputs[0].text)
    # is what comes after the "输出：" prompt.
    model_full_output_text = "输出： [认同逻辑" + output_obj.outputs[0].text

    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    match = output_marker_pattern.search(model_full_output_text)
    if match:
        # Extract the content after the "输出[:：]\s*" marker
        # This is essentially output_obj.outputs[0].text, but with leading/trailing whitespace potentially handled by \s*
        extracted_label_part = match.group(1).strip()
        
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: Potentially **LABEL** or just LABEL
        # If it's **LABEL**, this will keep the asterisks. This might be desired or handled later.
        # The main goal here is to capture the label content.
        else:
            label = extracted_label_part.strip() # .strip() handles if extracted_label_part was just spaces
        
        # Check if the label is empty after stripping wrappers (e.g., ```\n \n``` or ` ` or if model output was empty)
        if not label:
            current_parsed_item["RL_Types"] = "PARSE_ERROR_EMPTY_LABEL"
            print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i}.")
            print(f"Problematic Raw Output (after marker search, before stripping):\n'{match.group(1)}'\n" + "="*30)
            print(f"Original model output part (output_obj.outputs[0].text):\n'{output_obj.outputs[0].text}'\n" + "="*30)

        else:
            current_parsed_item["RL_Types"] = label
            
    else:
        # Log a warning if the output marker is not found
        # The marker string in the warning is a human-readable representation of what was expected.
        print(f"Warning: Could not find the output marker pattern '输出[:：]\\s*(.*)' in model output for original text at index {i}.")
        print(f"Problematic Raw Output (model_full_output_text):\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename
# Assuming model_path is defined in a previous cell.
# Changed suffix from "_thinking" to "_final_label" to reflect the new parsing focus.
output_csv_filename = f"results/{model_path.split('/')[-1]}_no_cot.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())


Successfully processed 483 model outputs.
Results saved to gemma-3-27b-it_no_cot.csv

First 5 rows of the parsed data:
                                 Original_Input_Text  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那...   
1  真的很多，我就是周边的，不管现实还是网上看的很多，他们天天说自己土著然后说我们弯脚杆，碰到过...   
2  我是都江堰的，其他地方的我不敢说，但是我可是从小被成都口音嘲笑哦，我小时候去亲戚家耍过暑假，...   
3  那我说都江堰话被笑的更多，我以前被介绍了个成都人，然后天天给我说他家以前二环以内的，说他家以...   
4  那证明成都人的确不咋地，本来成都话口音离普通话语区就够偏了，很多怪音，他们还歧视本省的人，无...   

                                           RL_Types  \
0               [认同逻辑1：Vernacular Spatial Authority]   
1  [认同逻辑1, 认同逻辑4]\n\n**解释：**\n\n*   **认同逻辑1：Verna...   
2            [认同逻辑4：Linguistic-Cultural Recognition]   
3            [认同逻辑4：Linguistic-Cultural Recognition]   
4            [认同逻辑4：Linguistic-Cultural Recognition]   

                                    Raw_Model_Output  
0         输出： [认同逻辑1：Vernacular Spatial Authority]\n  
1  输出： [认同逻辑1, 认同逻辑4]\n\n**解释：**\n\n*   **认同逻辑1：V...  
2      输出： [认同逻辑4：Linguistic-Cultural Recog

In [24]:
results_df['RL_Types']

0                   [认同逻辑1：Vernacular Spatial Authority]
1      [认同逻辑1, 认同逻辑4]\n\n**解释：**\n\n*   **认同逻辑1：Verna...
2                [认同逻辑4：Linguistic-Cultural Recognition]
3                [认同逻辑4：Linguistic-Cultural Recognition]
4                [认同逻辑4：Linguistic-Cultural Recognition]
                             ...                        
478                                       [认同逻辑1, 认同逻辑3]
479    [认同逻辑1：Vernacular Spatial Authority, 认同逻辑3：Fam...
480    [认同逻辑1：Vernacular Spatial Authority, 认同逻辑4：Lin...
481    [认同逻辑1：Vernacular Spatial Authority, 认同逻辑2：Adm...
482                 [认同逻辑1：Vernacular Spatial Authority]
Name: RL_Types, Length: 483, dtype: object